##필요한 라이브러리 및 함수 로드

In [1]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [3]:
def load_entire_place(db):
  cursor= db.cursor()
  df = pd.DataFrame()
  
  load_place_sql= (f"SELECT place_id, region2 FROM places_analysis")
  
  cursor.execute(load_place_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['id','region2']

  return df

In [4]:
def load_entire_course(db):
  cursor= db.cursor()
  df = pd.DataFrame()
  
  load_course_sql= (f"SELECT id, places FROM course_ ")

  cursor.execute(load_course_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['id','places']

  return df

##location 정의 및 적용

In [5]:
east = ['구좌','성산','표선','우도']
west = ['안덕','대정','한경','한림','추자도','가파도','마라도','비양도','차귀도']
north = ['제주시내','애월','조천']
south = ['서귀포시내','남원','중문']

In [6]:
db = load_db()
course = load_entire_course(db)

In [7]:
course.head(3)

,id,places
0,1,"[602, 10, 58, 29]"
1,2,"[19, 37, 10, 602, 60, 58, 29]"
2,3,"[19, 602, 10, 58, 29]"


In [8]:
place = load_entire_place(db)

In [9]:
place.head(4)

,id,region2
0,204,추자도
1,542,추자도
2,591,추자도
3,592,추자도


In [10]:
sample_course = [19, 37, 10, 602, 60, 58, 29]

In [11]:
location_template = [east,west,south,north]

In [12]:
location_template

[['구좌', '성산', '표선', '우도'],
 ['안덕', '대정', '한경', '한림', '추자도', '가파도', '마라도', '비양도', '차귀도'],
 ['서귀포시내', '남원', '중문'],
 ['제주시내', '애월', '조천']]

In [13]:
for crs in course['places']:
  print(crs)
  break

[602, 10, 58, 29]


##여행지 위치정보 로드

In [14]:
entire_direction_point = []
for crs in tqdm(course['places']):
  direction_point = np.zeros(4)
  for location in crs:
    region = place.loc[place['id']==location,'region2'].values[0]
    #region_list.append(region)
    for i in range(4):
      if region in location_template[i]:
        direction_point[i] += 1
  entire_direction_point.append(direction_point)  

100%|██████████| 3811/3811 [00:20<00:00, 181.63it/s]


In [15]:
direction_df = pd.DataFrame(entire_direction_point)
direction_df.columns = ['east','west','south','north']

##코스에 위치정보 반영

In [16]:
tmp_np = np.array([1,4,2,0])
max_idx = tmp_np.argmax()
percentage = tmp_np[max_idx]/tmp_np.sum()
tmptest = f'{percentage : 0.2f}'
print(tmptest)
print(type(tmptest))

 0.57
<class 'str'>


In [17]:
save_max_location = []
for crs in entire_direction_point:
  max_idx = crs.argmax()
  max_ratio = crs[max_idx] / crs.sum()
  if max_ratio > 0.5:
    if max_idx == 0:
      save_max_location.append("East")
    elif max_idx==1:
      save_max_location.append("West")
    elif max_idx==2:
      save_max_location.append("South")
    else:
      save_max_location.append("North")
  else:
    save_max_location.append("Mixed")
  #max_ratio = f"{max_ratio:.2f}"
  #print(crs, max_ratio,max_idx,save_max_location[cnt])

In [18]:
save_max_location[:20]

['Mixed',
 'Mixed',
 'Mixed',
 'North',
 'Mixed',
 'Mixed',
 'Mixed',
 'Mixed',
 'West',
 'East',
 'West',
 'East',
 'Mixed',
 'Mixed',
 'Mixed',
 'Mixed',
 'Mixed',
 'North',
 'Mixed',
 'Mixed']

In [19]:
course['main_location'] = save_max_location

In [20]:
course

,id,places,main_location
0,1,"[602, 10, 58, 29]",Mixed
1,2,"[19, 37, 10, 602, 60, 58, 29]",Mixed
2,3,"[19, 602, 10, 58, 29]",Mixed
3,4,"[37, 202, 58]",North
4,5,"[37, 60, 202, 58, 29]",Mixed
...,...,...,...
3806,3807,"[59, 8, 5, 3, 439, 6, 22, 10, 24, 15, 9]",Mixed
3807,3808,"[37, 144, 8, 78, 456, 66, 62, 49, 162, 5, 88, ...",Mixed
3808,3809,"[18, 15, 87, 29, 9, 5, 179, 112, 61]",South
3809,3810,"[22, 62, 5, 75, 284, 142, 172]",Mixed


In [21]:
direction_df

,east,west,south,north
0,2.0,1.0,0.0,1.0
1,2.0,2.0,0.0,3.0
2,2.0,1.0,0.0,2.0
3,1.0,0.0,0.0,2.0
4,2.0,1.0,0.0,2.0
...,...,...,...,...
3806,4.0,4.0,2.0,1.0
3807,5.0,3.0,6.0,4.0
3808,2.0,1.0,5.0,1.0
3809,0.0,3.0,2.0,2.0


In [22]:
joined_course = course.join(direction_df)
joined_course

,id,places,main_location,east,west,south,north
0,1,"[602, 10, 58, 29]",Mixed,2.0,1.0,0.0,1.0
1,2,"[19, 37, 10, 602, 60, 58, 29]",Mixed,2.0,2.0,0.0,3.0
2,3,"[19, 602, 10, 58, 29]",Mixed,2.0,1.0,0.0,2.0
3,4,"[37, 202, 58]",North,1.0,0.0,0.0,2.0
4,5,"[37, 60, 202, 58, 29]",Mixed,2.0,1.0,0.0,2.0
...,...,...,...,...,...,...,...
3806,3807,"[59, 8, 5, 3, 439, 6, 22, 10, 24, 15, 9]",Mixed,4.0,4.0,2.0,1.0
3807,3808,"[37, 144, 8, 78, 456, 66, 62, 49, 162, 5, 88, ...",Mixed,5.0,3.0,6.0,4.0
3808,3809,"[18, 15, 87, 29, 9, 5, 179, 112, 61]",South,2.0,1.0,5.0,1.0
3809,3810,"[22, 62, 5, 75, 284, 142, 172]",Mixed,0.0,3.0,2.0,2.0


###최종 결과본

In [23]:
joined_course[['id','places','east','west','south','north','main_location']]

,id,places,east,west,south,north,main_location
0,1,"[602, 10, 58, 29]",2.0,1.0,0.0,1.0,Mixed
1,2,"[19, 37, 10, 602, 60, 58, 29]",2.0,2.0,0.0,3.0,Mixed
2,3,"[19, 602, 10, 58, 29]",2.0,1.0,0.0,2.0,Mixed
3,4,"[37, 202, 58]",1.0,0.0,0.0,2.0,North
4,5,"[37, 60, 202, 58, 29]",2.0,1.0,0.0,2.0,Mixed
...,...,...,...,...,...,...,...
3806,3807,"[59, 8, 5, 3, 439, 6, 22, 10, 24, 15, 9]",4.0,4.0,2.0,1.0,Mixed
3807,3808,"[37, 144, 8, 78, 456, 66, 62, 49, 162, 5, 88, ...",5.0,3.0,6.0,4.0,Mixed
3808,3809,"[18, 15, 87, 29, 9, 5, 179, 112, 61]",2.0,1.0,5.0,1.0,South
3809,3810,"[22, 62, 5, 75, 284, 142, 172]",0.0,3.0,2.0,2.0,Mixed


In [26]:
joined_course = joined_course[['id','east','west','south','north','main_location']]

##Db에 적용

In [27]:
for info in joined_course.values:
  id=info[0]
  east = info[1]
  west = info[2]
  south = info[3]
  north = info[4]
  main_location = info[5]
  for i in info:
    print(i)
  break
#sql = f"update table course_ set east={east},west={west},south={south},'north={north},main_location={main_location} where id = {id}"

1
2.0
1.0
0.0
1.0
Mixed


##sample 뽑아서 확인 - id : 115번, 봄을 찾아 떠나는 제주도 동쪽여행 3박 4일 - 테스트

In [52]:
tttt = np.zeros(4)
for i in [8,13,4,26,145,3,3,52]:
  print(place.loc[place['id']==i,'region2'].values[0])
  for j in range(4):
    if place.loc[place['id']==i,'region2'].values[0] in location_template[j]:
      tttt[j] += 1

한림
구좌
구좌
구좌
성산
성산
성산
성산


In [55]:
tttt

array([7., 1., 0., 0.])

##추천 메소드에 인자로 받기

In [98]:
str1 = "North,East"
str1.split(",")

['North', 'East']

In [ ]:
direction_list = ['North','East']
num_of_direction = len(direction_list) 
#기존 sql where 절 내에 location_sql 정보 추가하자


#case 1 : 모든 지역 선택했거나 아무지역도 선택안하면 -> 전지역 
if num_of_direction == 4 or num_of_direction == 0:
  location_sql = 'and 1=1'
#case 2 : 한 지역만 선택한 경우 
elif num_of_direction == 1:
  direction = direction_list[0]
  location_sql = f"and main_location = {direction}"
#case 3 : 여러 지역을 선택한 경우
else:
  if num_of_direction == 2:
    location_sql = f"and main_location = Mix and {direction_list[0]} >0 and {direction_list[1]} >0"
  elif num_of_direction == 3:
    location_sql = f"and main_location = Mix and {direction_list[0]} >0 and {direction_list[1]} >0 and {direction_list[2]} >0"


In [34]:
sql = "select * from course_"
cursor = db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[(1, '[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+아쿠아플라넷', [602, 10, 58, 29], 1.25, 2.0, 1.25, 1.25, 2.75, 1.0, 1.75, 0.25, 3, 4), (2, '[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커스+아쿠아플라넷+오션아레나', [19, 37, 10, 602, 60, 58, 29], 1.7142857142857142, 2.4285714285714284, 1.2857142857142858, 1.4285714285714286, 2.571428571428572, 0.5714285714285714, 2.2857142857142856, 0.4285714285714285, 3, 7), (3, '[부산출발_※3일차 석식 전복죽포함※]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+아쿠아플라넷', [19, 602, 10, 58, 29], 1.4, 2.2, 1.2, 1.4, 2.8, 0.8, 2.0, 0.2, 3, 5), (4, '[청주출발]"모두 알뜰 패키지" 제주 관광호텔 2박3일 비밀의 그림자+ 웰빙 족욕', [37, 202, 58], 2.0, 3.0, 1.6666666666666667, 1.6666666666666667, 1.6666666666666667, 1.0, 3.0, 0.6666666666666666, 6, 3), (5, '[청주출발]◈감귤타르트증정/코시롱한나들이◈아쿠아플라넷+유람선-제주 로베로관광호텔 2박3일 산방산유람선+아쿠아플라넷', [37, 60, 202, 58, 29], 1.6, 2.4, 1.6, 1.4, 2.0, 0.8, 2.8, 0.6, 3, 5), (6, '[청주출발]◈감귤타르트증정/名品 효도◈고품격 노팁 노옵션-제주 로베로관광호텔 2박3일 해저잠수함+워터서커스+전일정식사포함', [37, 10, 65, 118, 58, 9], 2.5, 2.6666666666666665, 1.8333333333333

In [37]:
result = pd.DataFrame(result)
result.columns = ['id','title','places','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','length']
result.head(2)

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length
0,1,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"[602, 10, 58, 29]",1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1.750000,0.250000,3,4
1,2,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,"[19, 37, 10, 602, 60, 58, 29]",1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3,7


In [39]:
joined_course.head(2)

,id,east,west,south,north,main_location
0,1,2.0,1.0,0.0,1.0,Mixed
1,2,2.0,2.0,0.0,3.0,Mixed


In [42]:
join_result = result.merge(joined_course,on='id',how='inner')

In [43]:
join_result

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location
0,1,[청주출발]연합패키지 제주를 반하다 3일_관광호텔 화순곶자왈+서커스+석예원한방족욕+...,"[602, 10, 58, 29]",1.250000,2.000000,1.250000,1.250000,2.750000,1.000000,1.750000,0.250000,3,4,2.0,1.0,0.0,1.0,Mixed
1,2,[청주출발]연합패키지 노팁/노옵션 제주 3일_관광호텔 산방산유람선+ 화순곶자왈+서커...,"[19, 37, 10, 602, 60, 58, 29]",1.714286,2.428571,1.285714,1.428571,2.571429,0.571429,2.285714,0.428571,3,7,2.0,2.0,0.0,3.0,Mixed
2,3,[부산출발_※3일차 석식 전복죽포함※]연합패키지 제주를 반하다 3일_관광호텔 화순곶...,"[19, 602, 10, 58, 29]",1.400000,2.200000,1.200000,1.400000,2.800000,0.800000,2.000000,0.200000,3,5,2.0,1.0,0.0,2.0,Mixed
3,4,"[청주출발]""모두 알뜰 패키지"" 제주 관광호텔 2박3일 비밀의 그림자+ 웰빙 족욕","[37, 202, 58]",2.000000,3.000000,1.666667,1.666667,1.666667,1.000000,3.000000,0.666667,6,3,1.0,0.0,0.0,2.0,North
4,5,[청주출발]◈감귤타르트증정/코시롱한나들이◈아쿠아플라넷+유람선-제주 로베로관광호텔 2...,"[37, 60, 202, 58, 29]",1.600000,2.400000,1.600000,1.400000,2.000000,0.800000,2.800000,0.600000,3,5,2.0,1.0,0.0,2.0,Mixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,3807,제주도,"[59, 8, 5, 3, 439, 6, 22, 10, 24, 15, 9]",2.454545,2.636364,1.454545,1.363636,1.818182,0.909091,2.090909,0.181818,1,11,4.0,4.0,2.0,1.0,Mixed
3807,3808,Jeju!,"[37, 144, 8, 78, 456, 66, 62, 49, 162, 5, 88, ...",2.000000,2.444444,1.500000,1.277778,1.388889,1.166667,2.666667,1.000000,10,18,5.0,3.0,6.0,4.0,Mixed
3808,3809,제주도여행,"[18, 15, 87, 29, 9, 5, 179, 112, 61]",1.444444,2.000000,1.333333,1.000000,1.666667,1.111111,2.444444,0.444444,9,9,2.0,1.0,5.0,1.0,South
3809,3810,제주도!,"[22, 62, 5, 75, 284, 142, 172]",1.000000,1.000000,0.857143,1.000000,1.142857,0.857143,1.714286,1.571429,2,7,0.0,3.0,2.0,2.0,Mixed


In [44]:
join_result.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.09_course.csv")

##DB에 반영

In [56]:
from tqdm import tqdm

In [57]:
db= load_db()
cursor = db.cursor()

In [63]:
db.rollback()

In [64]:
for crs in tqdm(join_result.values):
  cursor.execute("insert into course_(title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location) VALUES(%s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s)",(crs[1],crs[2],crs[3],crs[4],crs[5],crs[6],crs[7],crs[8],crs[9],crs[10],crs[11],crs[12],crs[13],crs[14],crs[15],crs[16],crs[17]))
db.commit()

100%|██████████| 3811/3811 [11:08<00:00,  5.70it/s]


##Lambda에 적용

In [28]:
import json
import pandas as pd
import psycopg2
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

def load_db():
  endpoint = os.environ['END_POINT']
  dbname = os.environ['DB_NAME']
  user = os.environ['USER_NAME']
  password = os.environ['WONSEOK']
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result


def path_divider(day,path):
  path_per_day = []
  checker = day

  length = len(path)
  print('length :',length )

  remainder = length%day
  quotient = length//day

  start = 0
  check=0

  while start<length:
    print(path[start:start + quotient + check])
    path_per_day.append(path[start:start + quotient + check])
    start += quotient+check
    checker-=1
    if checker<=remainder:
      check = 1

  return path_per_day

In [123]:
def load_course(day,db,include,location):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  length = len(include)
  count = 0
  location_count = len(location)
  if location_count == 4 or location_count == 0:
    location_sql = ' and 1=1'
  #case 2 : 한 지역만 선택한 경우 
  elif location_count == 1:
    location_sql = f" and main_location = \'{location[0]}\'"
  #case 3 : 여러 지역을 선택한 경우
  else:
    if location_count == 2:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0"
    elif location_count == 3:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0 and {location[2]} >0"


  df = pd.DataFrame()
  while length >= count:
    possible_cases = list(combinations(include,length-count))
    ##sql query 날리고 결과 받아서
    for now in possible_cases:
      print("now : ",list(now))
      now_include = list(now)
      load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places, main_location\
 FROM course_ WHERE length>={min_length} and length<={max_length} and array{now_include}::smallint[] <@ places" + location_sql)
      #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
      
      cursor.execute(load_course_sql)
      result = cursor.fetchall()

      #현재 포함시킬 place를 모두 가지는 코스가 없는경우 
      if len(result) == 0:
        continue
      #현재 Place를 모두 포함하는 코스가 있는경우
      else:
        print("현재 now를 포함하는 df : \n",pd.DataFrame(result),"\n****************************\n")
        df = pd.concat([df, pd.DataFrame(result)])
    
    #dataframe에 값이 존재하면 (최적의 코스를 찾았다면)
    if not df.empty:
      break    
    count += 1
  
  #해당 조건에 만족하는 코스가 없는 경우
  if df.empty:
    print("no course")
    return df

  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places','main_location']

  return df
  #intersect 밖의 place를 모든 df의 Places list에 append 해줘야함.

In [124]:
def recommend_by_theme(user_id, day, include_list,location,db):
  top = 5
  course = load_course(day,db,include_list,location)
  if course.empty:
    print("empty course")
    return None

  user_taste = calculate_taste(user_id,db)
  user_taste = user_taste.reshape(1,-1)

  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  print(user_taste)
  
  similarity = cosine_similarity(user_taste, course[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)

  for i in range(-1,-(len(result_index)+1),-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  #결과는 가장 일치율이 높은거 1개의 index만 던져주자
  result = top_N_result[0]
  result_df = course.iloc[result]
  print(result_df)
  
  not_included_places = list(set(include_list) - set(result_df['places']))
  if 0 not in include_list:
    result_df['places'] += not_included_places
  
  path = path_divider(day,result_df['places'])
  print('path : ',path)
  return path
  

In [126]:
my_location = ['East','West','South','North']
duration = 4
id= 184
include = [3,17,45]

In [127]:
check_result = recommend_by_theme(user_id = id, day=duration, include_list=include, location = my_location,db=db)

now :  [3, 17, 45]
now :  [3, 17]
현재 now를 포함하는 df : 
        0                         1         2         3         4         5   \
0      77             깐수&꼉이의 제주 여행기  2.200000  2.300000  1.600000  1.300000   
1     147  25만원으로 혼자 떠난 제주도 3박4일 여행  2.533333  3.000000  2.000000  2.000000   
2     149            2박 3일 제주 기본 일정  2.312500  2.375000  1.375000  1.312500   
3     193     ?제주 뚜벅이 버스여행? ???=333  2.384615  2.769231  1.615385  1.923077   
4     206          처자들끼리 버스로 제주 누비기  2.363636  2.636364  1.454545  1.454545   
..    ...                       ...       ...       ...       ...       ...   
122  3657                       제주도  2.384615  2.538462  1.538462  1.076923   
123  3665                     제주도여행  2.562500  2.937500  1.750000  1.187500   
124  3693       2015.04.25~27 제주도여행  2.272727  2.727273  1.545455  1.454545   
125  3707                         ★  2.500000  2.875000  2.125000  1.125000   
126  3760                 설레이는 제주여행  2.363636  2.545455  1.909091  1.272727  

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [122]:
print(check_result)

[[5, 38, 16], [17, 61, 2], [310, 134, 336, 3], [52, 13, 25, 45]]


In [88]:
check_result

[[80, 13], [30, 219], [76, 10, 38], [22, 174, 8]]

In [95]:
join_result[join_result['id']==2566]

,id,title,places,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,length,east,west,south,north,main_location
2565,2566,혜리랑 제주도,"[25, 223, 707, 317, 379, 91, 4, 42, 179, 223, ...",2.125,2.6875,1.625,1.6875,2.25,1.25,2.75,0.0625,5,16,7.0,4.0,1.0,4.0,Mixed


##코스 확인용 df 변환 메소드 : df를 받으면 places id리스트를 title 리스트로 변환해주는 역할


In [96]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/07.31_attraction_db.csv",index_col=0)

In [97]:
attraction.sample()

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
692,693,법화사,제주특별자치도 서귀포시 하원북로 35번길 15-28,제주 천년고찰의 법화사이다.,33.265095,126.45783,064-738-5225,관광지,서귀포시,서귀포시내,...,2,1,0,1,0,0,3,2,12,문화관광


In [104]:
def id_to_title(course_df,attraction_df):
  places = course_df['places'].values[0]
  title_list = []
  for i in places:
    now = attraction_df.loc[attraction_df['id']==i,'title']
    title_list.append(now)
  return title_list
  

In [105]:
tt = id_to_title(join_result[join_result['id']==2566],attraction)

In [108]:
attraction.loc[attraction['id']==223]

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,nature,outdoor,fatigue,sea,walking,exciting,day,culture,group,tag
222,223,이호테우해수욕장,제주특별자치도 제주시 도리로 20,붉은 조랑말과 흰 조랑말의 극명하게 대조되는 색감이 이국적인 풍경을 선사하는 해변,33.496185,126.455952,064-728-4927,관광지,제주시,제주시내,...,3,3,2,3,2,2,2,0,9,"자연경관,포토스팟,해변,휴식/힐링,여름"


[24    김녕해수욕장
 Name: title, dtype: object, 222    이호테우해수욕장
 Name: title, dtype: object, 706    월정리해안도로
 Name: title, dtype: object, 316    종달리해안도로
 Name: title, dtype: object, 378    법환포구
 Name: title, dtype: object, 90    아날로그감귤밭
 Name: title, dtype: object, 3    비자림
 Name: title, dtype: object, 41    제주김녕미로공원
 Name: title, dtype: object, 178    이니스프리제주하우스
 Name: title, dtype: object, 222    이호테우해수욕장
 Name: title, dtype: object, 12    월정리해변
 Name: title, dtype: object, 45    김경숙해바라기농장
 Name: title, dtype: object, 8    섭지코지
 Name: title, dtype: object, 173    성이시돌목장
 Name: title, dtype: object, 294    낙천리아홉굿의자마을
 Name: title, dtype: object, 9    카멜리아힐
 Name: title, dtype: object]

###08.08 lambda

In [ ]:
#08.08 lambda
import json
import pandas as pd
import psycopg2
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

def load_db():
  endpoint = os.environ['END_POINT']
  dbname = os.environ['DB_NAME']
  user = os.environ['USER_NAME']
  password = os.environ['WONSEOK']
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result


def path_divider(day,path):
  path_per_day = []
  checker = day

  length = len(path)
  print('length :',length )

  remainder = length%day
  quotient = length//day

  start = 0
  check=0

  while start<length:
    print(path[start:start + quotient + check])
    path_per_day.append(path[start:start + quotient + check])
    start += quotient+check
    checker-=1
    if checker<=remainder:
      check = 1

  return path_per_day


def load_course(day,db,include):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  length = len(include)
  count = 0
  
  df = pd.DataFrame()
  while length >= count:
    possible_cases = list(combinations(include,length-count))
    ##sql query 날리고 결과 받아서
    for now in possible_cases:
      print("now : ",list(now))
      now_include = list(now)
      load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places\
 FROM course_ WHERE length>={min_length} and length<={max_length} and array{now_include}::smallint[] <@ places")
      #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
      
      cursor.execute(load_course_sql)
      result = cursor.fetchall()

      #현재 포함시킬 place를 모두 가지는 코스가 없는경우 
      if len(result) == 0:
        continue
      #현재 Place를 모두 포함하는 코스가 있는경우
      else:
        print("현재 now를 포함하는 df : \n",pd.DataFrame(result),"\n****************************\n")
        df = pd.concat([df, pd.DataFrame(result)])
    
    #dataframe에 값이 존재하면 (최적의 코스를 찾았다면)
    if not df.empty:
      break    
    count += 1

  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
  #intersect 밖의 place를 모든 df의 Places list에 append 해줘야함.




def recommend_by_theme(user_id, day, include_list,db):
  top = 5
  course = load_course(day,db,include_list)
  user_taste = calculate_taste(user_id,db)
  user_taste = user_taste.reshape(1,-1)

  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  print(user_taste)
  
  similarity = cosine_similarity(user_taste, course[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)

  for i in range(-1,-(len(result_index)+1),-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  #결과는 가장 일치율이 높은거 1개의 index만 던져주자
  result = top_N_result[0]
  result_df = course.iloc[result]
  print(result_df)
  
  not_included_places = list(set(include_list) - set(result_df['places']))
  if 0 not in include_list:
    result_df['places'] += not_included_places
  
  path = path_divider(day,result_df['places'])
  print('path : ',path)
  return path
  
def lambda_handler(event, context):

  id = int(event['queryStringParameters']['id'])
  day = int(event['queryStringParameters']['day'])
  include = list(map(int,event['queryStringParameters']['include'].split(',')))
  
  try:
    db = load_db()
  except:
        return {
            "success": False,
            "message": "Database Error",
        }
  result_path = recommend_by_theme(id, day,include,db)
  
  

  return {
      'statusCode': 200,
      'body': json.dumps({  
                            "courseList" : result_path,
                            })
  }
  